In [2]:
import pandas as pd 

In [3]:
df = pd.read_csv("books_train.csv")
df.head(5)


,title,description,categories
0,ساختار نظریه‌های جامعه‌شناسی ایران,«ساختار نظریه‌های جامعه‌شناسی ایران» نوشته ابو...,جامعه‌شناسی
1,جامعه و فرهنگ کانادا,«جامعه و فرهنگ کانادا» از مجموعه کتاب‌های «جام...,جامعه‌شناسی
2,پرسش از موعود,پرسش‌های مختلفی درباره زندگی و شخصیت امام مهدی...,کلیات اسلام
3,موج، دریا,«موج دریا» به قلم مهری ماهوتی (-۱۳۴۰) و تصویرگ...,داستان کودک و نوجوانان
4,پرسش از غرب,«پرسش از غرب» به قلم دکتر اسماعیل شفیعی سروستا...,جامعه‌شناسی


In [53]:
from __future__ import unicode_literals
from hazm import *

normalizer = Normalizer()
stemmer = Stemmer()

In [51]:
normalizer.normalize(df['title'][0])

'ساختار نظریه\u200cهای جامعه\u200cشناسی ایران'

In [59]:
stemmer.stem('رفتم')

'رف'

In [38]:
word_tokenize(df['title'][1])

['جامعه', 'و', 'فرهنگ', 'کانادا']

In [5]:
df['categories'].value_counts()

کلیات اسلام               425
داستان کودک و نوجوانان    425
جامعه‌شناسی               425
رمان                      425
داستان کوتاه              425
مدیریت و کسب و کار        425
Name: categories, dtype: int64

In [6]:
ignore = ['»', '«', 'و', ',', 'را', 'از', '.', 'به',  '،', ':', '/']
ignore

['»', '«', 'و', ',', 'را', 'از', '.', 'به', '،', ':', '/']

In [28]:
novel = dict()
sociology = dict()
islamic = dict()
short_story = dict()
kids = dict()
business = dict()

In [64]:
novel.clear()
sociology.clear()
islamic.clear()
short_story.clear()
kids.clear()
business.clear()

for index, row in df.iterrows():
    tokens = word_tokenize(row['title'])
    #print(row['categories'])
    for token in tokens:
        word = token
        if (not word in ignore):
            #print(word)
            cat = row['categories']
            
            if (cat == 'رمان'):
                novel[word] = novel.get(word, 0) + 1
            if (cat == 'جامعه‌شناسی'):
                sociology[word] = sociology.get(word, 0) + 1
            if (cat == 'کلیات اسلام'):
                islamic[word] = islamic.get(word, 0) + 1
            if (cat == 'داستان کوتاه'):
                short_story[word] = short_story.get(word, 0) + 1
            if (cat == 'داستان کودک و نوجوانان'):
                kids[word] = kids.get(word, 0) + 1
            if (cat == 'مدیریت و کسب و کار'):
                business[word] = business.get(word, 0) + 1 



In [ ]:
p_novel = dict()
p_sociology = dict()
p_islamic = dict()
p_short_story = dict()
p_kids = dict()
p_business = dict()

By p_novel[x] here we mean $P(x | novel)$ which is the probability of seeing the word $x$ in a novel.